In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Qt5Agg')
%matplotlib qt
from scipy.optimize import curve_fit
import glob
import re
from tkinter import filedialog as fd
from tkinter import *

In [ ]:
def getFilePathFromDialog(_initialdir):
    root = Tk()
    FilePath = fd.askopenfilename(initialdir = _initialdir)
    root.destroy()
    return FilePath

def getDirectoryPathFromDialog(_initialdir):
    root = Tk()
    Directory = fd.askdirectory(initialdir = _initialdir)
    root.destroy()
    return Directory

In [ ]:
def readFile(path, skiprows):
    return np.loadtxt(path, dtype=float, skiprows=skiprows)

def readFileFromDialog(skiprows):
    return readFile(getFilePathFromDialog(dataPath),skiprows)

In [ ]:
def readParticleConfig(path,ConfigurationNumber,Skiplines):
    TotalNumberOfParticles = int(re.search(r'(?<=N=)[\d]*',path).group())
    with open(path,'r') as F:
        Data = F.read()
    RowOffset = Skiplines+ConfigurationNumber*(TotalNumberOfParticles+2)
    print("RowOffset: ",RowOffset)
    Data = Data.strip()
    FirstRow = Data.split(sep='\n')[RowOffset]
    NumberOfAParticles = int(FirstRow.split()[7].strip('|'))
    NumberOfBParticles = int(FirstRow.split()[9].strip('|'))
    ListOfRows = Data.split(sep='\n')[RowOffset+1:RowOffset+1+TotalNumberOfParticles]
    if (len(ListOfRows) > 0):
        NumberOfColumns = len(ListOfRows[0].split()) - 1
    else:
        NumberOfColumns = 0
    APositions = np.empty((NumberOfAParticles, 2))
    AVelocities = np.empty((NumberOfAParticles, 2))
    BPositions = np.empty((NumberOfBParticles, 2))
    BVelocities = np.empty((NumberOfBParticles, 2))
    AIndex = 0
    BIndex = 0
    
    for i in range(len(ListOfRows)):
        if (ListOfRows[i].split()[NumberOfColumns] == 'A'):
            for j in range(2):
                APositions[AIndex,j] = ListOfRows[i].split()[j]
            for j in range(2):
                AVelocities[AIndex,j] = ListOfRows[i].split()[2+j]
            AIndex += 1
        else:
            for j in range(2):
                BPositions[BIndex,j] = ListOfRows[i].split()[j]
            for j in range(2):
                BVelocities[BIndex,j] = ListOfRows[i].split()[2+j]
            BIndex += 1
    return APositions, AVelocities, BPositions, BVelocities

def readParticleConfigFromDialog(ConfigurationNumber,Skiplines):
    return readParticleConfig(getFilePathFromDialog(dataPath),ConfigurationNumber,Skiplines)

In [ ]:
def LinearFunction(x,m,n):
    return m*x+n

In [ ]:
def extractAngles(structureFactorData):
    angleList = []
    numberOfEntriesList = []
    currentAngle = structureFactorData[0,0]
    angleList.append(currentAngle)
    numberOfEntries=0
    for angle in structureFactorData[:,0]:
        if (angle != currentAngle):
            angleList.append(angle)
            currentAngle = angle
            numberOfEntriesList.append(numberOfEntries)
            numberOfEntries=1
        else:
            numberOfEntries+=1
    numberOfEntriesList.append(numberOfEntries)
    return angleList,numberOfEntriesList

In [ ]:
def extractSeparateAngleArrays(structureFactorData):
    angleList, numberOfEntriesList = extractAngles(structureFactorData)
    angleArrayList = []
    currentIndex = 0
    for i in range(len(angleList)):
        angleArrayList.append(structureFactorData[currentIndex:currentIndex+numberOfEntriesList[i],1:])
        currentIndex+=numberOfEntriesList[i]
    return angleList,angleArrayList

In [ ]:
def plotStructureFactors(angleList,angleArrayList,angleIndex):
    fig, ax = plt.subplots(4,1,squeeze=False,figsize=(10,12), dpi = 100)
    labels = ['$S_{AA}$','$S_{BB}$','$S_{AB}$','$S_{cc}$']
    fig.suptitle('angle={}'.format(angleList[angleIndex]))
    for i in range(4):
        ax[i,0].plot(angleArrayList[angleIndex][:,0],angleArrayList[angleIndex][:,i+1])
        ax[i,0].set_ylabel(labels[i])
        ax[i,0].set_xlabel('k')
    fig.tight_layout()

In [ ]:
dataPath = getDirectoryPathFromDialog('')

## Energy series

In [ ]:
EnergySeries = readFileFromDialog(1)

In [ ]:
Fig, Ax = plt.subplots(2,2,squeeze=False,figsize=(15,9))
Ax[0,0].plot(EnergySeries[:1000,2],label='H=U+T')
Ax[0,0].plot(EnergySeries[:1000,0],label='U')
Ax[0,0].plot(EnergySeries[:1000,1],label='T')
Ax[0,0].legend()
Ax[0,1].plot(EnergySeries[:,2],label='H=U+T')
Ax[0,1].set_ylabel('H=T+U')
MeanH = np.mean(EnergySeries[:,2])
print("H=",MeanH,'+-',np.std(EnergySeries[:,2]))
print("U=",np.mean(EnergySeries[:,0]),'+-',np.std(EnergySeries[:,0]))
print("T=",np.mean(EnergySeries[:,1]),'+-',np.std(EnergySeries[:,1]))
#Ax[0,1].plot(MeanH*np.ones(len(EnergySeries[:,2])))
n,bins,patches = Ax[1,1].hist(EnergySeries[:,2],bins = 41, density = True)
#Ax[1,1].vlines(MeanH,0,np.max(n),color='darkorange')
Ax[1,1].set_title("Histogram of H")
Ax[1,0].plot(EnergySeries[:,1],color='g')
Ax[1,0].set_ylabel('T')

In [ ]:
PotEnergySeriesMC = readFileFromDialog(1)

In [ ]:
Fix, Ax = plt.subplots(2,1,squeeze=False,figsize=(10,12))
Ax[0,0].plot(PotEnergySeriesMC)
Ax[0,0].plot(EnergySeries[:,0])
print("U Monte Carlo = ",np.mean(PotEnergySeriesMC),"+-",np.std(PotEnergySeriesMC))
print("U MD = ",np.mean(EnergySeries[:,0]),"+-",np.std(EnergySeries[:,0]))
a = Ax[1,0].hist(PotEnergySeriesMC,bins = 41, density = True, alpha = 0.7)
a = Ax[1,0].hist(EnergySeries[:,0],bins = 41, density = True,alpha =0.7)

## Average velocity

In [ ]:
avgVelocities = readFileFromDialog(0)

In [ ]:
Fig, Ax = plt.subplots(1,2,squeeze=False,figsize=(10,5), dpi = 100)
Ax[0,0].plot(avgVelocities[1],avgVelocities[0],'.')
Ax[0,0].set_xlabel('avg $v_x$ [$\sigma/\\tau$]')
Ax[0,0].set_ylabel('y / $\sigma$')
popt, pcov = curve_fit(LinearFunction, avgVelocities[1], avgVelocities[0])
print('shear rate: {:.3E}'.format(1/popt[0]))
Ax[0,0].plot(avgVelocities[1],LinearFunction(avgVelocities[1],*popt))
Ax[0,0].set_title('Average $v_x$ vs y')
Ax[0,0].ticklabel_format(axis='x', style='sci', scilimits=(-2,2))

Ax[0,1].plot(avgVelocities[2],avgVelocities[0],'.')
Ax[0,1].set_xlabel('avg $v_y$ [$\sigma/\\tau$]')
Ax[0,1].set_ylabel('y / $\sigma$')
Ax[0,1].set_title('Average $v_y$ vs y')
Ax[0,1].ticklabel_format(axis='x', style='sci', scilimits=(-2,2))

#Fig.savefig("plots/shear_analysis/avg_delta_x_field_no_lees_edwards_T=1.png",format='png')

## Stress plotting

In [ ]:
shearStressSeries = readFileFromDialog(1)
print(np.mean(shearStressSeries))

In [ ]:
Fig, Ax = plt.subplots(1,2, squeeze=False, figsize=(12,5))
Ax[0,0].plot(shearStressSeries[:],alpha=0.4)
Ax[0,0].set_xlabel('timestep')
Ax[0,0].set_ylabel('shear stress')
Ax[0,0].set_title('shear stress vs time')
#Ax[0,0].set_xscale('log')
Ax[0,0].hlines(np.mean(shearStressSeries), 0, len(shearStressSeries), color='orange')
#Fig.savefig("shear_stress_time_series_shear_rate=0.05.png",format='png')
n,bins,patches = Ax[0,1].hist(shearStressSeries[:],bins = 41, density = True)
Ax[0,1].vlines(np.mean(shearStressSeries),0,np.max(n),color='darkorange')
Ax[0,1].set_title('shear stress histogram')
Ax[0,1].set_xlabel('shear stress')

In [ ]:
NumberOfSubdivisions = 14

In [ ]:
xStressData = readFileFromDialog(2)

In [ ]:
Fig, Ax = plt.subplots(NumberOfSubdivisions,2,squeeze=False, figsize= (10,2.5*NumberOfSubdivisions),dpi =100)
xAxis = np.linspace(1.0/(2.0*NumberOfSubdivisions),1.0-1.0/(2.0*NumberOfSubdivisions),num = NumberOfSubdivisions)
for i in range(NumberOfSubdivisions):
    Ax[i,0].plot(xAxis,xStressData[NumberOfSubdivisions-1-i::NumberOfSubdivisions,3],label='y={}'.format(xStressData[NumberOfSubdivisions-1-i,0]))
    Ax[i,0].plot(xAxis,np.ones(len(xAxis))*np.mean(xStressData[NumberOfSubdivisions-1-i::NumberOfSubdivisions,3]))
    Ax[i,0].legend()
    Ax[i,1].plot(xAxis,xStressData[NumberOfSubdivisions-1-i::NumberOfSubdivisions,4],label='y={}'.format(xStressData[NumberOfSubdivisions-1-i,0]))
    MeanShear = np.mean(xStressData[NumberOfSubdivisions-1-i::NumberOfSubdivisions,4])
    Ax[i,1].plot(xAxis,np.ones(len(xAxis))*MeanShear,label='Avg = {:.4f}'.format(MeanShear))
    Ax[i,1].legend()
Fig.suptitle("Stresses of cuts in x direction (left column normal stresses, right column shear stresses)",y= 0.90)

In [ ]:
Mean = 0
for i in range(NumberOfSubdivisions-1):
    Mean += np.mean(xStressData[NumberOfSubdivisions-i-2::NumberOfSubdivisions,4])
print("Avg shear stress except top layer: {:.4f}".format(Mean/(NumberOfSubdivisions-1)))

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize= (10,6),dpi =100)
xAxis = np.linspace(1.0/(2.0*NumberOfSubdivisions),1.0-1.0/(2.0*NumberOfSubdivisions),num = NumberOfSubdivisions)
for i in range(NumberOfSubdivisions):
    line = Ax[0,0].plot(xAxis,xStressData[NumberOfSubdivisions-1-i::NumberOfSubdivisions,4])
    MeanShear = np.mean(xStressData[NumberOfSubdivisions-1-i::NumberOfSubdivisions,4])
    Ax[0,0].plot(xAxis,np.ones(len(xAxis))*MeanShear,color = line[0].get_color(), label='y={}, Avg = {:.4f}'.format(xStressData[NumberOfSubdivisions-1-i,0],MeanShear))
Ax[0,0].legend()

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize= (10,6),dpi =100)
xAxis = np.linspace(1.0/NumberOfSubdivisions,1.0,num = NumberOfSubdivisions)
for i in range(NumberOfSubdivisions):
    line = Ax[0,0].plot(xAxis,xStressData[NumberOfSubdivisions*i:NumberOfSubdivisions*(i+1),4])
    MeanShear = np.mean(xStressData[NumberOfSubdivisions*i:NumberOfSubdivisions*(i+1),4])
    Ax[0,0].plot(xAxis,np.ones(len(xAxis))*MeanShear,color = line[0].get_color(),label='x={}'.format(xAxis[i]))
Ax[0,0].legend()
Ax[0,0].set_xlim(0,1)

In [ ]:
yStressData = readFileFromDialog(2)

In [ ]:
Fig, Ax = plt.subplots(NumberOfSubdivisions,2,squeeze=False, figsize= (10,2.5*NumberOfSubdivisions),dpi =100)
xAxis = np.linspace(1.0/(2.0*NumberOfSubdivisions),1.0-1.0/(2.0*NumberOfSubdivisions),num = NumberOfSubdivisions)
for i in range(NumberOfSubdivisions):
    Ax[i,0].plot(xAxis,yStressData[i::NumberOfSubdivisions,3],label='x={}'.format(yStressData[i,0]))
    MeanNormal = np.mean(yStressData[i::NumberOfSubdivisions,3])
    Ax[i,0].plot(xAxis,np.ones(len(xAxis))*MeanNormal)
    Ax[i,0].legend()
    Ax[i,1].plot(xAxis,yStressData[i::NumberOfSubdivisions,4],label='x={}'.format(yStressData[i,0]))
    MeanShear = np.mean(yStressData[i::NumberOfSubdivisions,4])
    Ax[i,1].plot(xAxis,np.ones(len(xAxis))*MeanShear,label='Avg={:.3f}'.format(MeanShear))
    Ax[i,1].legend()
Fig.suptitle("Stresses of cuts in y direction (left column normal stresses, right column shear stresses)",y= 0.90)

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize= (10,6),dpi =100)
xAxis = np.linspace(1.0/(2.0*NumberOfSubdivisions),1.0-1.0/(2.0*NumberOfSubdivisions),num = NumberOfSubdivisions)
for i in range(NumberOfSubdivisions):
    line = Ax[0,0].plot(xAxis,yStressData[NumberOfSubdivisions-1-i::NumberOfSubdivisions,4])
    MeanShear = np.mean(yStressData[NumberOfSubdivisions-1-i::NumberOfSubdivisions,4])
    Ax[0,0].plot(xAxis,np.ones(len(xAxis))*MeanShear,color = line[0].get_color(), label='x={}, Avg = {:.4f}'.format(yStressData[NumberOfSubdivisions-1-i,0],MeanShear))
Ax[0,0].legend()

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize= (10,6),dpi =100)
xAxis = np.linspace(1.0/NumberOfSubdivisions,1.0,num = NumberOfSubdivisions)
for i in range(NumberOfSubdivisions):
    line = Ax[0,0].plot(xAxis,yStressData[NumberOfSubdivisions*i:NumberOfSubdivisions*(i+1),4])
    MeanShear = np.mean(yStressData[NumberOfSubdivisions*i:NumberOfSubdivisions*(i+1),4])
    Ax[0,0].plot(xAxis,np.ones(len(xAxis))*MeanShear,color = line[0].get_color(),label='y={}'.format(xAxis[i]))
#Ax[0,0].legend()
Ax[0,0].set_xlim(0,1)

In [ ]:
print("Avg normal stress perp to x direction: {:.4f}".format(np.mean(xStressData[:,3])))
print("Avg normal stress perp to y direction: {:.4f}".format(np.mean(yStressData[:,3])))
print("Avg pressure: {:.4f}".format((np.mean(xStressData[:,3])+np.mean(yStressData[:,3]))*0.5))
print("Avg shear stress x direction: {:.4f}".format(np.mean(xStressData[:,4])))
print("Avg shear stress y direction: {:.4f}".format(np.mean(yStressData[:,4])))

## Shear stresses vs shear rates

In [ ]:
ShearRates = np.array([0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.10])
ShearStresses = np.array([0.0067,0.0193,0.0340,0.0405,0.0537,0.0635,0.0732,0.0827,0.0935,0.1051])

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False, figsize=(6,5), dpi = 100)
Ax[0,0].plot(ShearRates, ShearStresses,'.')
Ax[0,0].set_xlabel(r'shear rate '+r'[$1/\tau_0$]')
Ax[0,0].set_ylabel('shear stress '+r'[$\epsilon/\sigma^2$]')
popt, pcov = curve_fit(lambda x,m: LinearFunction(x,m,0.0), ShearRates, ShearStresses)
Ax[0,0].legend()
print('viscosity= {:.4f}'.format(popt[0]), 'epsilon*tau_0 / (sigma^2)')
#Fig.savefig("avg_shear_stresses_vs_shear_rate_T=2.0.png",format='png')

## Number of particles

In [ ]:
NAHistequi = readFileFromDialog(1)

In [ ]:
NAHistshear = []
for i in range(5):
    newHistshear = readFileFromDialog(1)
    NAHistshear.append(newHistshear)

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False,figsize=(6,6), dpi = 100)
Ax[0,0].plot(NAHistequi[:,0],NAHistequi[:,1],label='equilibrium')
for i in range(5):
    Ax[0,0].plot(NAHistshear[i][:,0],NAHistshear[i][:,1])
Ax[0,0].legend()
#Ax[0,0].set_xlim(0.1,0.16)
#Ax[0,0].set_ylim(2.5,4.5)
Ax[0,0].set_xlabel('$x_A$')
Ax[0,0].set_ylabel('probability density')

## Binder cumulant and first moment vs shear rate

In [ ]:
cumulant_vs_shear_rate = readFileFromDialog(1)

In [ ]:
Fig, Ax = plt.subplots(1,2,squeeze=False,figsize=(12,6), dpi = 100)
Ax[0,0].plot(cumulant_vs_shear_rate[:,0],cumulant_vs_shear_rate[:,1])
Ax[0,1].plot(cumulant_vs_shear_rate[:,0],cumulant_vs_shear_rate[:,2])
Ax[0,0].set_xlabel('$\dot{\gamma}$')
Ax[0,1].set_xlabel('$\dot{\gamma}$')
Ax[0,0].set_ylabel('first moment xA')
Ax[0,1].set_ylabel('binder cumulant')

## Im(g_22)

In [ ]:
img22 = readFileFromDialog(1)

In [ ]:
Fig, Ax = plt.subplots(3,1,squeeze=False, figsize= (10, 12), dpi = 100)
labelList = ['Im($g_{22}^{AA}$)','Im($g_{22}^{BB}$)','Im($g_{22}^{AB}$)']
for i in range(3):
    Ax[i,0].plot(img22[:,0],img22[:,i+1])
    Ax[i,0].set_ylim(-0.4,0.3)
    Ax[i,0].set_xlabel('r [$\sigma$]')
    Ax[i,0].set_ylabel(labelList[i])
Fig.tight_layout()
#Fig.savefig("Img22_shear.png",format='png')

## structure factors

In [ ]:
structureFactors = readFileFromDialog(1)

In [ ]:
angleList,angleArrayList = extractSeparateAngleArrays(structureFactors)

In [ ]:
plotStructureFactors(angleList,angleArrayList,0)

In [ ]:
plotStructureFactors(angleList,angleArrayList,10)

## Avg MSD

In [ ]:
AvgMSDSeries = readFileFromDialog(0)

In [ ]:
Fig, Ax = plt.subplots(1,2,squeeze=False, figsize = (12,5), dpi = 100)
xdata = np.arange(1,len(AvgMSDSeries)+1)*stepsize
AvgMSD_Over_t_squared = AvgMSDSeries/(xdata*xdata)
Ax[0,0].plot(xdata,AvgMSD_Over_t_squared,'.')
popt, pcov = curve_fit(lambda x,n: LinearFunction(x,0.0,n), xdata[:9], AvgMSD_Over_t_squared[:9])
print(popt)
Ax[0,0].plot(xdata[:9],LinearFunction(xdata[:9],0.0,*popt))
Ax[0,0].set_xscale('log')
Ax[0,0].set_yscale('log')
Ax[0,0].set_xlabel('$t / \\tau$')
Ax[0,0].set_ylabel('$MSD/t^2$')

Ax[0,1].plot(xdata,AvgMSDSeries/xdata)
Ax[0,1].set_xscale('log')
Ax[0,1].set_yscale('log')
Ax[0,1].set_xlabel('$t / \\tau$')
Ax[0,1].set_ylabel('$MSD/t$')

## Plot state

In [ ]:
APos, AVel, BPos, BVel = readParticleConfigFromDialog(0,0)

In [ ]:
Fig, Host = plt.subplots(1,1,squeeze=False,figsize=(5,5),dpi =200)
Host[0,0].plot(APos[:,0],APos[:,1],'.',color='b',ms=4)
Host[0,0].plot(BPos[:,0],BPos[:,1],'.',color='darkorange',ms=4)
Host[0,0].set_xlabel('x')
Host[0,0].set_ylabel('y')
Host[0,0].set_xlim(0,1)
Host[0,0].set_ylim(0,1)

In [ ]:
Fig, Host = plt.subplots(1,1,squeeze=False,figsize=(5,5),dpi =200)
Host[0,0].plot(36.5*AVel[:,0],36.5*APos[:,1],'.',color='b',ms=4)
Host[0,0].plot(36.5*BVel[:,0],36.5*BPos[:,1],'.',color='darkorange',ms=4)
Host[0,0].set_ylabel('y')
Host[0,0].set_xlabel('$v_x$')
#Host[0,0].set_xlim(0,1)
#Host[0,0].set_ylim(0,1)

In [ ]:
Fig, Host = plt.subplots(1,1,squeeze=False,figsize=(5,5),dpi =200)
Host[0,0].plot(36.5*AVel[:,1],36.5*APos[:,1],'.',color='b',ms=4)
Host[0,0].plot(36.5*BVel[:,1],36.5*BPos[:,1],'.',color='darkorange',ms=4)
Host[0,0].set_ylabel('y')
Host[0,0].set_xlabel('$v_y$')
#Host[0,0].set_xlim(0,1)
#Host[0,0].set_ylim(0,1)